In [1]:
import tensorflow as tf

# Explicitly allow memory growth for GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')
        # Allow memory growth to avoid allocating all GPU memory at once
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("GPU configured successfully")
    except RuntimeError as e:
        print(e)


GPU configured successfully


In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("twitter_sentiment.csv")
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [4]:
len(df)

75681

In [5]:
df=df.drop(['2401','Borderlands'],axis=1)

In [6]:
df.columns=['review','reviewText']
df

,review,reviewText
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
75676,Irrelevant,⭐️ Toronto is the arts and culture capital of ...
75677,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
75678,Positive,Today sucked so it’s time to drink wine n play...
75679,Positive,Bought a fraction of Microsoft today. Small wins.


In [7]:
df.isnull().sum()

review          0
reviewText    686
dtype: int64

In [8]:
df.dropna(inplace=True)
len(df)

74995

In [9]:
df = df[df['reviewText'].apply(len)>1]
len(df)

74646

In [10]:
df['review'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [11]:
df['review'].value_counts()

review
Negative      22530
Positive      20843
Neutral       18285
Irrelevant    12988
Name: count, dtype: int64

In [12]:
df=df[df['review']!='Irrelevant']
df=df[df['review']!='Neutral']

In [13]:
len(df),df['review'].value_counts()

(43373,
 review
 Negative    22530
 Positive    20843
 Name: count, dtype: int64)

In [14]:
df=pd.get_dummies(df, columns=['review'])
df

,reviewText,review_Negative,review_Positive
0,I am coming to the borders and I will kill you...,False,True
1,im getting on borderlands and i will kill you ...,False,True
2,im coming on borderlands and i will murder you...,False,True
3,im getting on borderlands 2 and i will murder ...,False,True
4,im getting into borderlands and i can murder y...,False,True
...,...,...,...
75673,guess i'll broke.,False,True
75674,Please explain how this is possible! How can t...,True,False
75675,Good on Sony. As much as I want to see the new...,False,True
75678,Today sucked so it’s time to drink wine n play...,False,True


In [15]:
#pos=0,neg=1
df=df.drop(['review_Positive'],axis=1)
df['review_Negative']= df['review_Negative'].astype(int)
df

,reviewText,review_Negative
0,I am coming to the borders and I will kill you...,0
1,im getting on borderlands and i will kill you ...,0
2,im coming on borderlands and i will murder you...,0
3,im getting on borderlands 2 and i will murder ...,0
4,im getting into borderlands and i can murder y...,0
...,...,...
75673,guess i'll broke.,0
75674,Please explain how this is possible! How can t...,1
75675,Good on Sony. As much as I want to see the new...,0
75678,Today sucked so it’s time to drink wine n play...,0


In [16]:
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [17]:
## Lower All the cases
df['reviewText']=df['reviewText'].str.lower()
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',str(x)))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

C:\Users\aneet\AppData\Local\Temp\ipykernel_11768\338547949.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())


In [18]:
df

,reviewText,review_Negative
0,coming borders kill,0
1,im getting borderlands kill,0
2,im coming borderlands murder,0
3,im getting borderlands 2 murder,0
4,im getting borderlands murder,0
...,...,...
75673,guess ill broke,0
75674,please explain possible let companies overchar...,1
75675,good sony much want see new ps5 whats going ri...,0
75678,today sucked time drink wine n play borderland...,0


In [19]:
from nltk.stem import WordNetLemmatizer
lm=WordNetLemmatizer()

In [20]:
def lemmatize_words(text):
    return " ".join([lm.lemmatize(word,pos='v') for word in text.split()])

In [21]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))
df

,reviewText,review_Negative
0,come border kill,0
1,im get borderlands kill,0
2,im come borderlands murder,0
3,im get borderlands 2 murder,0
4,im get borderlands murder,0
...,...,...
75673,guess ill break,0
75674,please explain possible let company overcharge...,1
75675,good sony much want see new ps5 whats go right...,0
75678,today suck time drink wine n play borderlands ...,0


In [22]:
y=df.drop(['reviewText'],axis=1)
y

,review_Negative
0,0
1,0
2,0
3,0
4,0
...,...
75673,0
75674,1
75675,0
75678,0


In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],y,
                                              test_size=0.20)

In [24]:
len(X_train)

34698

In [25]:
import gensim
from gensim.models import KeyedVectors,Word2Vec
import gensim.downloader as api
wv=api.load("word2vec-google-news-300")

In [26]:
def preprocess(sentences, wv):
    vectors = []
    valid_indices = []
    for i, sentence in enumerate(sentences):
        tokens = sentence.split()  # Simple tokenization
        sentence_vector = []
        for token in tokens:
            if token in wv:
                sentence_vector.append(wv[token])
        if sentence_vector:  # Avoid empty vectors
            vectors.append(np.mean(sentence_vector, axis=0))
            valid_indices.append(i)
    return np.array(vectors, dtype='float32'), valid_indices

In [27]:
X_train_vectors, train_valid_indices  = preprocess(X_train.tolist(), wv)
X_test_vectors , test_valid_indices = preprocess(X_test.tolist(), wv)

In [28]:
X_train_vectors[0]

array([ 4.11028191e-02,  4.08935547e-02,  6.51463121e-02,  7.08182231e-02,
        2.53121518e-02,  4.90679070e-02, -3.56619693e-02, -8.01914781e-02,
        1.14955358e-01,  6.42288774e-02, -5.34493569e-03, -8.10198113e-02,
       -5.09033203e-02, -1.58952978e-02, -1.08799525e-01,  3.33077572e-02,
       -1.99759342e-02,  1.51279999e-03,  3.61328125e-02, -9.39810649e-02,
        2.06996370e-02, -2.24609375e-02,  1.14257812e-01,  5.08161262e-02,
        1.15661621e-01,  7.51604373e-03, -6.01981021e-02, -7.08007812e-03,
        1.04980469e-02, -3.96641307e-02,  1.80576872e-02,  2.28445865e-02,
       -8.95647332e-02, -2.09045410e-02, -8.68530273e-02, -3.79289896e-03,
        3.83232646e-02,  2.51464844e-02, -2.02636719e-02,  1.26708984e-01,
        6.45228773e-02, -3.42930369e-02,  1.69189453e-01, -1.68871190e-02,
       -1.09654017e-01, -6.94056926e-03, -1.21442519e-01, -7.67299086e-02,
       -4.15736623e-02,  4.01960090e-02, -6.09654002e-02,  1.37642995e-01,
        4.96303029e-02,  

In [29]:
len(X_train_vectors),len(X_test_vectors)

(33910, 8515)

In [42]:
len(X_train),len(X_test)

(34698, 8675)

In [43]:
len(y_train),len(y_test)

(34698, 8675)

In [32]:
Y_train = y_train['review_Negative'].tolist()
Y_test = y_test['review_Negative'].tolist()
Y_train = [y_train[i] for i in train_valid_indices]
Y_test = [y_test[i] for i in test_valid_indices]

In [33]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB().fit(X_train_vectors,Y_train)

In [35]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [36]:
y_pred=model.predict(X_test_vectors)

In [37]:
print("accuracy: ",accuracy_score(Y_test,y_pred))

accuracy:  0.7100411039342337


In [38]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_vectors, Y_train)


RandomForestClassifier(random_state=42)

In [40]:
y_pred_rf= rf_classifier.predict(X_test_vectors)
accuracy = accuracy_score(Y_test, y_pred_rf)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.8979448032883147


In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 50
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

vocab_size = 5000
embedding_dim = 64

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(32),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_padded, y_train, epochs=10, batch_size=32)


Epoch 1/10
1085/1085 [==============================] - 15s 12ms/step - loss: 0.6888 - accuracy: 0.5255
Epoch 2/10
1085/1085 [==============================] - 12s 11ms/step - loss: 0.5898 - accuracy: 0.7208
Epoch 3/10
1085/1085 [==============================] - 11s 10ms/step - loss: 0.5762 - accuracy: 0.7427
Epoch 4/10
1085/1085 [==============================] - 11s 10ms/step - loss: 0.5569 - accuracy: 0.7583
Epoch 5/10
1085/1085 [==============================] - 11s 10ms/step - loss: 0.5536 - accuracy: 0.7614
Epoch 6/10
1085/1085 [==============================] - 11s 10ms/step - loss: 0.5482 - accuracy: 0.7665
Epoch 7/10
1085/1085 [==============================] - 12s 11ms/step - loss: 0.5428 - accuracy: 0.7713
Epoch 8/10
1085/1085 [==============================] - 12s 11ms/step - loss: 0.5287 - accuracy: 0.7830
Epoch 9/10
1085/1085 [==============================] - 12s 11ms/step - loss: 0.5230 - accuracy: 0.7870
Epoch 10/10
1085/1085 [==============================] - 12s 11m

In [32]:
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {accuracy}')

272/272 [==============================] - 2s 5ms/step - loss: 0.5626 - accuracy: 0.7517
Test Accuracy: 0.7517002820968628


In [39]:
# Build GRU model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

maxlen=50
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
model.add(GRU(128, return_sequences=True))
model.add(GRU(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

Epoch 1/15
976/976 [==============================] - 14s 12ms/step - loss: 0.6937 - accuracy: 0.5092 - val_loss: 0.6923 - val_accuracy: 0.5231
Epoch 2/15
976/976 [==============================] - 11s 12ms/step - loss: 0.6933 - accuracy: 0.5131 - val_loss: 0.6931 - val_accuracy: 0.5228
Epoch 3/15
976/976 [==============================] - 12s 12ms/step - loss: 0.6932 - accuracy: 0.5112 - val_loss: 0.6923 - val_accuracy: 0.5231
Epoch 4/15
976/976 [==============================] - 11s 12ms/step - loss: 0.6931 - accuracy: 0.5146 - val_loss: 0.6921 - val_accuracy: 0.5231
Epoch 5/15
976/976 [==============================] - 12s 12ms/step - loss: 0.6931 - accuracy: 0.5120 - val_loss: 0.6919 - val_accuracy: 0.5231
Epoch 6/15
976/976 [==============================] - 12s 12ms/step - loss: 0.6022 - accuracy: 0.6433 - val_loss: 0.3654 - val_accuracy: 0.8516
Epoch 7/15
976/976 [==============================] - 12s 12ms/step - loss: 0.2983 - accuracy: 0.8791 - val_loss: 0.2626 - val_accuracy: